In [ ]:
# @title Install dependencies
!pip install --upgrade bigframes google-cloud-aiplatform


In [ ]:
# @title Login into Google Cloud
gcloud auth application-default login

In [ ]:
#@title Setup Project Variables { run: "auto", display-mode: "form" }
location = "us" #@param {type:"string"}
project_id = "<your_project>" #@param {type:"string"}
dataset = "<your_dataset>" #@param {type:"string"}


In [ ]:
# @title Initialise BigFrames.Pandas
# With BigQuery DataFrames, you can use many familiar Pandas methods, but the
# processing happens BigQuery rather than the runtime, allowing you to work with larger
# DataFrames that would otherwise not fit in the runtime memory.
import bigframes.pandas as bf
bf.reset_session()
bf.options.bigquery.location = location #this variable is set based on the dataset you chose to query
bf.options.bigquery.project = project_id #this variable is set based on the dataset you chose to query and where your BigQuery environment

In [ ]:
#@title BigQuery ML Variables to call Gemini. Needed to create a connection to Gemini via Vertex
connection_name = "gemini-connection" #@param {type:"string"}
bqml_model = "gemini_model" #@param {type:"string"}

### Create a connection to Gemini via Vertex AI - at this point we can only do this via bq and utils commands
You can skip these steps if you already have configured a connection to Gemini as per:
https://cloud.google.com/bigquery/docs/generate-text#create_a_connection

In [ ]:
! bq mk --connection --location={location} --project_id={project_id} --connection_type=CLOUD_RESOURCE {connection_name}

/tools/google-cloud-sdk/platform/bq/third_party/requests/__init__.py:103: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (None)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Connection 774298971519.us-central1.gemini-connection successfully created


In [ ]:
# @title  We need to get the ServiceAccountId to give Vertex User permission to it
! bq show --connection {project_id}.{location}.{connection_name}

/tools/google-cloud-sdk/platform/bq/third_party/requests/__init__.py:103: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (None)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Connection csilvariverademo.us-central1.gemini-connection

                     name                      friendlyName   description    Last modified         type        hasCredential                                            properties                                            
 -------------------------------------------- -------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  774298971519.us-central1.gemini-connection                                22 Apr 09:16:09   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-774298971519-ql9l@gcp-sa-bigquery-condel.iam.g

In [ ]:
# copy the ServiceAccountId from the previous step - it should be in the form of xxxx@cp-sa-bigquery-condel.iam.gserviceaccount.com"
member= "xxxxxx@gcp-sa-bigquery-condel.iam.gserviceaccount.com"

In [ ]:
# Add permissions to call Gemini via Vertex
!gcloud projects add-iam-policy-binding {project_id} --member='serviceAccount:{member}' --role='roles/aiplatform.user' --condition=None

In [ ]:
# create the model in BigQuery
from bigframes.ml.llm import GeminiTextGenerator
import bigframes.pandas as bpd

session = bf.get_global_session()
# initialise variables
connection = f"{project_id}.{location}.{connection_name}"
model = GeminiTextGenerator(session=session, connection_name=connection)

model.to_gbq(model_name= f"{project_id}.{dataset}.{bqml_model}", replace = True)

GeminiTextGenerator(connection_name='csilvariverademo.us.bigframes_dev',
                    session=<bigframes.session.Session object at 0x7ab097c13ca0>)

# Machine Readable News

In [ ]:
# @title Get the sentiment for each Vodafone new ons via gemini 1.0 pro in BigQuery

from bigframes.ml.llm import GeminiTextGenerator
from IPython.display import Markdown


## A couple of things to highlight
## 1) LSEG provides their own sentiment. For this demo we wanted to try our own to
##.   showcase the connection from BigQuery to Gemini via VertexAI
##.2) At the moment BigQuery can only call Gemini pro 1.0 due to 1.5 not bein GA yet
##    this actually works well since we can easily work with multiple languages to summarise the news

# llm params
temperature = 0.6
max_output_tokens = 800
top_k =40
top_p = 0.9

# define your mrn_table. default is MRN_UNLIMITED
mrn_table = "MRN_UNLIMITED"

# For every record in the table, we will get the sentiment from Gemini Pro. Limiting to 100 records just to avoid long waits
mrn_query = """
 DECLARE query STRING;
  SET query = \"\"\"
  SELECT * FROM
  ML.GENERATE_TEXT(
    MODEL `{0}.{2}`,
    (
  SELECT
        CONCAT('You are helping an executive understanding the sentiment of the stock market news ',
        ' Given the headline and body of the news, classify the sentiment as positive or negative based and provide a summary of the news.',
        ' Always translate to english, mention the company legal name based on the RIC, the headline, the date, as well as the sentiment of the news.',
         'RIC:', RIC,
        ' headline: ', data.headline,
        ' body: ', data.body,
        ' date:', _created
        )
        AS prompt from `{0}.{1}.{3}`
        WHERE DATE(_Created) >= DATE('2022-09-06')
        AND DATE(_Created) <= DATE('2022-09-07')
        AND data.pubStatus <> 'stat:canceled'
        -- Filter for Vodafone RIC
        AND UPPER(ric )= 'VOD.L'
        AND data.body IS NOT NULL
        limit 100
        ),
    STRUCT(
      {4} AS temperature,
      {5} AS max_output_tokens,
      TRUE AS flatten_json_output))
  \"\"\";
  EXECUTE IMMEDIATE query;
""".format(project_id, dataset, bqml_model, mrn_table, temperature, max_output_tokens)
mrn_df = bf.read_gbq(mrn_query)
# Display the first row
display(Markdown(mrn_df['ml_generate_text_llm_result'].iloc[0]))

 **Company:** Vodafone Group Plc

**Headline:** Britain seeking new entrants to 5G market - PM's spokesman

**Date:** 2022-09-06 16:15:39.980+00

**Sentiment:** Neutral

**Summary:** The United Kingdom government is looking to diversify its 5G telecommunications market by seeking new entrants. The government is in talks with allies, including the United States, about potential new entrants. The government is also assessing the potential impact of new U.S. sanctions on Huawei.

In [ ]:
# @title Get all the news in one single string
import pandas as pd
cluster_info = []
# save the dataframe as pandas
pd_df = mrn_df.to_pandas()

# get the results as array
for i, row in pd_df.iterrows():
  cluster_info.append("{0}".format(row["ml_generate_text_llm_result"]))

# join the array as a single string
cluster_info_str = " ".join(cluster_info)
cluster_info_str


' **Company:** Vodafone Group Plc\n\n**Headline:** Britain seeking new entrants to 5G market - PM\'s spokesman\n\n**Date:** 2022-09-06 16:15:39.980+00\n\n**Sentiment:** Neutral\n\n**Summary:** The United Kingdom government is looking to diversify its 5G telecommunications market by seeking new entrants. The government is in talks with allies, including the United States, about potential new entrants. The government is also assessing the potential impact of new U.S. sanctions on Huawei.  **Company:** Vodafone Group PLC\n\n**Headline:** Google considering buying about 5% stake in Vodafone Idea: FT\n\n**Date:** 2022-09-06 16:15:39.980+00\n\n**Sentiment:** Positive\n\n**Summary:**\n- Google, a subsidiary of Alphabet, is considering investing in Vodafone Idea, the Indian subsidiary of British telecom giant Vodafone, as part of its strategy to expand its operations in India.\n- Google is looking at acquiring approximately 5% of Vodafone Idea\'s shares.  **Company:** Vodafone Group Plc (VOD.L

# Tick History

In [ ]:
# @title Get the VWAP for Vodafone

# Define the table. Normally it is the name of the venue
table = "LSE_NORMALISED"

query = ("""

 WITH AllTrades AS(
      SELECT Date_Time,RIC,Price,Volume
      FROM `{0}{1}.{2}`
      WHERE Price IS NOT NULL
      -- Specific Date/Time range:
      AND (Date_Time BETWEEN "2022-09-07 12:00:00.000000" AND "2022-09-12 12:59:59.999999")
      AND Type = "Trade"
      AND VOLUME > 0
      AND PRICE > 0
      # All trades reported as "On Book" & "Regular Trades"
      # This is according to the FIX specs, most European trading venues adhere to this
      AND RIGHT(REGEXP_EXTRACT(Qualifiers, r";(.*)\[MMT_CLASS\]"),14) LIKE "12%"
    )
  SELECT RIC, ROUND(SAFE_DIVIDE(SUM(Volume*Price),SUM(Volume)),3) AS VWAP,SUM(Volume) AS TotalVolume,COUNT(RIC) AS NumTrades, extract(DATE FROM Date_Time) AS vwap_date
  FROM AllTrades
  WHERE RIC IN ('VOD.L')
  GROUP BY RIC, vwap_date
  ORDER BY RIC

    """.format(project_id,dataset, table))
bq_df = bf.read_gbq(query)
bq_df.head(5)


,RIC,VWAP,TotalVolume,NumTrades,vwap_date
0,VOD.L,109.150000000,16892818.000000000,5350,2022-09-12
1,VOD.L,108.866000000,25612529.000000000,9642,2022-09-09
2,VOD.L,111.110000000,16319743.000000000,5008,2022-09-07
3,VOD.L,108.572000000,31895900.000000000,10093,2022-09-08


In [ ]:
# @title Get all the VWAPS in one single string
vwap_info = []
# save the dataframe as pandas
pd_df = bq_df.to_pandas()

# get the results as array
for i, row in pd_df.iterrows():
  vwap_info.append("""vwap {0}, Total Volume {1}, Number of trades {2}""".format(row["VWAP"],row["TotalVolume"],row["NumTrades"]))

# join the array as a single string
vwap_info_str = " ".join(vwap_info)


# Gemini 1.5 Call

In [ ]:
# @title: Call Gemini 1.5 using the 1M token window

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

# function to call Gemini 1.5
def generate():

  vertexai.init(project="csilvariverademo", location="us-central1")
  model = GenerativeModel("gemini-experimental")
  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )
  output = ""
  for response in responses:
    output = output + response.text
    #print(response.text, end="")
  display(Markdown(output))

# Prompt definiton
prompt = """You are helping an executive to understand the financial market news and if there's a correlation between the news and the stock price.
    First, You are going to receive a list of all the news which will include: Company, Headline, Sentiment and their Summaries
    Second, You will receive a table with the Volume Weitghed Average Price (VWAP), total volume and date for the 5 following days after the news.
    Given this information, make an executive summary including the sentiment, the company name, the date, what is the overall perception of the market
    and if there's a correlation between the news and the total volume trades happening that day.
    Always try to give an opinion on the correlation and why. Include as well the VWAP for the days
    <news>
    {0}
    </news>
    <table>
    {1}
    </table>
""".format(cluster_info_str, vwap_info_str)

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generate()

## Executive Summary: Vodafone Group Plc - September 6, 2022

**Overall Sentiment:** The news surrounding Vodafone Group Plc on September 6, 2022, presented a mixed bag of positive, negative, and neutral sentiment. 

**Key Themes:**

* **Merger and Acquisition Activity:**  The potential merger of O2 and Virgin Media in the UK dominated the news cycle, generating both positive and negative implications for Vodafone. While some analysts viewed the merger as a positive development for the industry, others expressed concerns about Vodafone's competitive position without a fixed-line partner.
* **5G Development:** News regarding 5G rollout and adoption showcased a blend of optimism and challenges. Vodafone's partnership with OPPO to accelerate 5G adoption and positive court rulings regarding 5G deployment in the Netherlands countered concerns about potential slowdowns in financing and investment due to the COVID-19 crisis. 
* **Financial Performance:** Vodafone's financial performance received positive attention, with reports highlighting resilient cash flow and the company's decision to maintain dividends despite the pandemic's impact. 
* **Leadership Changes:** The appointment of Jean-Francois Van Boxmeer as the new chairman was generally viewed as a positive development, bringing experienced leadership to the company.

**Market Perception and Correlation with Trading Volume:**

The market perception of Vodafone appears to be cautiously optimistic. While concerns exist regarding the competitive landscape and potential challenges in 5G deployment, the company's strong financial performance and strategic partnerships appear to have instilled confidence.

**Correlation Analysis:**

Analyzing the provided data, it's challenging to establish a direct and immediate correlation between the news sentiment and the total trading volume on September 6, 2022. The day exhibited a relatively average trading volume compared to the following days. However, the subsequent increase in trading volume, particularly on September 7th and 8th, could potentially be attributed to the news regarding the O2-Virgin Media merger and its implications for Vodafone. Investors might have reacted to the news, leading to increased trading activity. 

**VWAP Analysis:**

The VWAP remained relatively stable over the five days following the news, fluctuating within a narrow range. This suggests that the average price at which Vodafone shares were traded did not experience significant volatility in response to the news events.

**Recommendations:**

* **Monitor the O2-Virgin Media merger closely:**  Assess the potential impact on Vodafone's competitive position and explore potential strategic partnerships or acquisitions to strengthen its fixed-line offerings.
* **Continue investing in 5G development and deployment:** Advocate for policies that facilitate 5G rollout and leverage partnerships to accelerate adoption and explore new use cases. 
* **Maintain focus on financial performance and shareholder value:**  Continue delivering strong cash flow, managing debt effectively, and ensuring attractive returns for investors. 
* **Effectively communicate the company's strategy and vision:**  Clearly articulate Vodafone's response to industry challenges and opportunities to maintain investor confidence and market support. 


In [ ]:
# @title Bonus, Explain VWAP changes with Gemini
from bigframes.ml.llm import GeminiTextGenerator
from IPython.display import Markdown

# llm params
temperature = 0.9
max_output_tokens = 800
top_k =40
top_p = 0.9

query = """
  DECLARE query STRING;
  SET query = \"\"\"
  SELECT * FROM
  ML.GENERATE_TEXT(
    MODEL `{0}.{2}`,
    (
  SELECT
        CONCAT('From the following trading information, make an executive summary comparing vwap, volume and number of trades over 2 days.',
        'Always mention the company legal name based on ' ,
        'RIC:', RIC,
        ' vwap day one: ',VWAP_2023_05_16,
        ' vwap day two: ', VWAP_2023_05_17,
        ' total trades day one: ', NumTrades_2023_05_16,
        ' trades day two:', NumTrades_2023_05_17,
        ' total volume day one: ',TotalVolume_2023_05_16,
        ' total volume day two: ',TotalVolume_2023_05_17 )
        AS prompt from `{0}.{1}.LSE_PIVOTED_BY_RIC`
        WHERE RIC = 'VOD.L'
        ),
    STRUCT(
      {3} AS temperature,
      {4} AS max_output_tokens,
      TRUE AS flatten_json_output))
  \"\"\";
  EXECUTE IMMEDIATE query;
""".format(project_id,dataset,bqml_model, temperature, max_output_tokens)
gemini_df = bf.read_gbq(query)
display(Markdown(gemini_df['ml_generate_text_llm_result'].iloc[0]))
# gemini_df


 ## Vodafone Group Plc (VOD.L) Trading Summary

**Date: [Insert Date]**

**Executive Summary**

Over the course of two trading days, Vodafone Group Plc (VOD.L) experienced significant changes in its volume-weighted average price (VWAP), trading volume, and the number of trades. 

**VWAP**

On day one, the VWAP for VOD.L was 82.722, while on day two, it increased to 83.998, representing a rise of 1.53%. This increase indicates a slight overall upward trend in the stock's average price over the two-day period.

**Volume**

The total volume traded for VOD.L on day one was 74,181,849, which significantly exceeded the volume traded on day two, which was 20,159,511. This marked a significant decrease of approximately 72.8% in trading activity from day one to day two.

**Number of Trades**

The number of trades executed on day one was 18,615, while on day two, it decreased to 7,023. This represents a substantial reduction of approximately 62.3% in trading activity based on the number of trades.

In conclusion, while VOD.L's VWAP showed a slight increase from day one to day two, there was a significant decline in both trading volume and the number of trades executed over the same period. This suggests a decrease in overall market participation and activity for VOD.L during the second day of trading.